# DataFrames.jl 介紹 (二): Joins 與 Split-Apply-Combine Strategy

## Day 018 作業1：The Split-Apply-Combine Strategy

同 Day 017，請載入 2020/4/2 COVID-19 Daily Report 資料集，計算每個國家/地區的累計確診人數，並列出所有資料筆數。

In [1]:
using DataFrames, CSV

In [2]:
df = CSV.read("../data/04-02-2020.csv")

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat
,Int64⍰,String⍰,String⍰,String,String,Float64⍰
1,45001,Abbeville,South Carolina,US,2020-04-02 23:25:27,34.2233
2,22001,Acadia,Louisiana,US,2020-04-02 23:25:27,30.2951
3,51001,Accomack,Virginia,US,2020-04-02 23:25:27,37.7671
4,16001,Ada,Idaho,US,2020-04-02 23:25:27,43.4527
5,19001,Adair,Iowa,US,2020-04-02 23:25:27,41.3308
6,29001,Adair,Missouri,US,2020-04-02 23:25:27,40.1906
7,40001,Adair,Oklahoma,US,2020-04-02 23:25:27,35.8849
8,8001,Adams,Colorado,US,2020-04-02 23:25:27,39.8743
9,16003,Adams,Idaho,US,2020-04-02 23:25:27,44.8933


計算計算每個國家/地區的累計確診人數

【提示】可使用 groupby 相關的函式

In [3]:
describe(df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,FIPS,30958.4,66,29093.0,99999
2,Admin2,,Abbeville,,unassigned
3,Province_State,,Alabama,,Zhejiang
4,Country_Region,,Afghanistan,,Zimbabwe
5,Last_Update,,2020-02-23 11:19:02,,2020-04-02 23:32:01
6,Lat,36.2992,-42.8821,37.7171,71.7069
7,Long_,-78.9577,-159.597,-86.5104,178.065
8,Confirmed,394.378,0,7.0,115242
9,Deaths,20.624,0,0.0,13915


In [4]:
# 顯示所有資料
sort(by(df, :Country_Region, :Confirmed => sum), :Confirmed_sum, rev=true)

,Country_Region,Confirmed_sum
,String,Int64
1,US,243453
2,Italy,115242
3,Spain,112065
4,Germany,84794
5,China,82432
6,France,59929
7,Iran,50468
8,United Kingdom,34173
9,Switzerland,18827


## 作業2：請列出美國各區域或事件 (依 Province_State 欄) 的累積確診、死亡、康復人數。

【提示】使用 Split-Apply-Combine Strategy 時，有幾種不同的函式可以使用，請參照 `DataFrames.jl` 官方文件 [https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby](https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby)

In [5]:
describe(df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,FIPS,30958.4,66,29093.0,99999
2,Admin2,,Abbeville,,unassigned
3,Province_State,,Alabama,,Zhejiang
4,Country_Region,,Afghanistan,,Zimbabwe
5,Last_Update,,2020-02-23 11:19:02,,2020-04-02 23:32:01
6,Lat,36.2992,-42.8821,37.7171,71.7069
7,Long_,-78.9577,-159.597,-86.5104,178.065
8,Confirmed,394.378,0,7.0,115242
9,Deaths,20.624,0,0.0,13915


In [6]:
func_tuple = (:Confirmed => sum, :Recovered => sum, :Deaths => sum)
us_df = filter(row -> row[:Country_Region] == "US", df)
by(us_df, :Province_State, func_tuple)

,Province_State,Confirmed_sum,Recovered_sum,Deaths_sum
,String⍰,Int64,Int64,Int64
1,South Carolina,1554,0,31
2,Louisiana,9159,0,310
3,Virginia,1706,0,41
4,Idaho,776,0,9
5,Iowa,614,0,11
6,Missouri,1857,0,22
7,Oklahoma,881,0,34
8,Colorado,3342,0,80
9,Illinois,7695,0,163
